# Les décorateurs

In [11]:
def sandwich(_func=None, *, hot=False):
    def deco(garniture):
        def wrapper(*args, **kargs):
            if hot:
                print('Chauffé')

            print("/TTTTTTTT\\")
            value = garniture(*args, **kargs)
            print("\\________/")

            return value + 2
        return wrapper

    if _func is None:
        return deco
    else:
        return deco(_func)

@sandwich()
def parisien():
    print("  Jambon ")
    print("  Beurre ")
    return 8

@sandwich
def lyonnais():
    print("  rosette ")
    print("  Beurre ")
    return 5

@sandwich(hot=True)
def kebab(sauce="blanche"):
    print(f'  {sauce}')
    print('  viande')
    print('  salade tomates oignons')
    return 4


In [12]:
print(parisien())

/TTTTTTTT\
  Jambon 
  Beurre 
\________/
10


In [13]:
lyonnais()

/TTTTTTTT\
  rosette 
  Beurre 
\________/


7

In [14]:
kebab('Samourai')

Chauffé
/TTTTTTTT\
  Samourai
  viande
  salade tomates oignons
\________/


6

In [27]:
kebab

<function __main__.sandwich.<locals>.wrapper()>

## Exercices
### Durée d'exécution

In [41]:
import time

def time_it(some_func):
    def wrapper(*args, **kargs):
        start = time.time()
        value = some_func(*args, **kargs)
        end = time.time()
        print(end - start, "secondes se sont écoulées")
        return value

    return wrapper

@time_it
def func():
    for x in range(1_000_000):
        y = x ** 2


In [48]:
func()

0.048175811767578125 secondes se sont écoulées


## Compteur

In [61]:
import functools

def count_calls(func):
    count = 0
    @functools.wraps(func)
    def inner():
        nonlocal count
        count += 1
        func()

    def inner_get_count():
        return count

    inner.nbcalls = inner_get_count


    return inner

@count_calls
def some_func():
    print("func called")

print(some_func.nbcalls())
some_func()
some_func()
print(some_func.nbcalls())

0
func called
func called
2


In [62]:
some_func

<function __main__.some_func()>

## Troisième exercice

In [19]:
import time
from collections import OrderedDict

def cachable(func):
    cached_values = OrderedDict()
    def inner(value:int):
        if value not in cached_values:
            result = func(value)
            cached_values[value] = result
            if len(cached_values) > 10:
                cached_values.popitem(last=False)
        else:
            result = cached_values[value]
            cached_values.move_to_end(value)

        return result
    return inner

In [22]:
@cachable
def long_call(value):
    time.sleep(2)
    return value**2


In [23]:
print(long_call(5))
print(long_call(6))
print(long_call(5))
print(long_call(7))

25
36
25
49


### Notifications

In [24]:
notifications = []

def register(_func=None, *, level:int=1):
    def register_inner(func):
        notifications.append((func, level))
        return func

    if _func is None:
        return register_inner
    else:
        return register_inner(_func)

@register(level=2)
def notify_mail():
    print("notification on mail")

def notify_sms():
    print("notification on message")

@register
def notify_push():
    print("notification on push service")

def send_notifications(level:int=2):
    for notification, notif_level in notifications:
        if notif_level >= level:
            notification()

In [25]:
send_notifications()

notification on mail
notification on push service
